In [1]:
import numpy as np
import plotly.graph_objects as go
matplotlib_style = 'fivethirtyeight' # ['fivethirtyeight', 'bmh', 'ggplot', 'seaborn', 'default', 'Solarize_Light2', 'classic', 'dark_background', 'seaborn-colorblind', 'seaborn-notebook']
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
%matplotlib inline
notebook_screen_res = 'retina' # ['retina', 'png', 'jpeg', 'svg', 'pdf']
%config InlineBackend.figure_format = notebook_screen_res

[**Runge-Kutta**](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) method implementation to solve [**SIR**](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology) differential equations system.

The **SIR model** is one of the simplest compartmental models, and many models are derivatives of this basic form. The model consists of three compartments: **S** for the number of susceptible, **I** for the number of infectious, and **R** for the number of recovered or deceased (or immune) individuals. This model is reasonably predictive for infectious diseases which are transmitted from human to human, and where recovery confers lasting resistance.

The functions governing the differential equations and a 4rth order Runge-Kutta method are:

In [2]:
# Susceptible equation
def fa(N, a, b, beta):
    fa = -beta*a*b
    return fa

# Infectious equation
def fb(N, a, b, beta, gamma):
    fb = beta*a*b - gamma*b
    return fb

# Removed(Recovered+Deceased) equation
def fc(N, b, gamma):
    fc = gamma*b
    return fc

# Runge-Kutta method of 4rth order for 3 dimensions (susceptible a, infectious b and removed c)
def rk4(N, a, b, c, fa, fb, fc, beta, gamma, hs):
    a1 = fa(N, a, b, beta)*hs
    b1 = fb(N, a, b, beta, gamma)*hs
    c1 = fc(N, b, gamma)*hs
    ak = a + a1*0.5; bk = b + b1*0.5; ck = c + c1*0.5
    a2 = fa(N, ak, bk, beta)*hs
    b2 = fb(N, ak, bk, beta, gamma)*hs
    c2 = fc(N, bk, gamma)*hs
    ak = a + a2*0.5; bk = b + b2*0.5; ck = c + c2*0.5
    a3 = fa(N, ak, bk, beta)*hs
    b3 = fb(N, ak, bk, beta, gamma)*hs
    c3 = fc(N, bk, gamma)*hs
    ak = a + a3; bk = b + b3; ck = c + c3
    a4 = fa(N, ak, bk, beta)*hs
    b4 = fb(N, ak, bk, beta, gamma)*hs
    c4 = fc(N, bk, gamma)*hs
    a = a + (a1 + 2*(a2 + a3) + a4)/6
    b = b + (b1 + 2*(b2 + b3) + b4)/6
    c = c + (c1 + 2*(c2 + c3) + c4)/6
    return a, b, c

Define the initial conditions and call the `rk4` method:

In [4]:
def SIR(N, b0, beta, gamma, hs):
    
    """
    N = total number of population
    beta = transition rate S->I
    gamma = transition rate I->R
    k =  denotes the constant degree distribution of the network (average value for networks in which 
    the probability of finding a node with a different connectivity decays exponentially fast)
    hs = jump step of the numerical integration
    """
    
    # Initial condition
    a = float(N-1)/N - b0
    b = float(1)/N + b0
    c = 0.

    sus, inf, rem = [], [], []
    for i in range(10000): # Run for a certain number of time-steps
        sus.append(a)
        inf.append(b)
        rem.append(c)
        a,b,c = rk4(N, a, b, c, fa, fb, fc, beta, gamma, hs)

    return sus, inf, rem

Results obtained for $N=0.3\cdot7.8\cdot10^9$ (30% of world population), only one initial infected case (constant $b_0=0$), transition rate `S->I` $\beta=0.25$ (or typical time between contacts $T_c=0.25$ days, or [basic reproduction number](https://en.wikipedia.org/wiki/Basic_reproduction_number) $R_0=4$ one person can infect 4 people per day), transition rate `I->R`  $\gamma=0.07$ (or typical time until recovery/decease $T_r=0.07$ days)  and $h_s=0.1$ (jump step of the numerical integration) are shown below:

In [8]:
# Parameters of the model
N = 0.3*7.8e9 # 30% of world population
b0 = 0 # one initial infected case

R0 = 4 # basic 
Tc = 1/R0 # typical time between contacts [days]  one person can infect 4 people per day
beta = 1/Tc # transition rate S->I

Tr = 14 # typical time until recovery/decease [days]
gamma = 1/Tr # transition rate I->R

hs = 0.1 # leap pass

susceptible, infectious, removed = SIR(N, b0, beta, gamma, hs)

# infectious + removed = confirmed cases (sigmoid)
infectrem = (np.array(infectious) + np.array(removed))*N
# growth rate of confirmed cases
casesgrowth = np.gradient(infectrem)

figC = go.Figure()

figC.add_trace(go.Scatter(y=np.array(susceptible)*N, mode='lines+markers', name='susceptible', marker=dict(size=3), 
                          line=dict(color='rgba(23,190,207,0.5)', width=1))) # blue-teal

figC.add_trace(go.Scatter(y=np.array(infectious)*N, mode='lines+markers', name='infectious', marker=dict(size=3), 
                          line=dict(color='rgba(214,39,40,0.5)', width=1))) # brick_red

figC.add_trace(go.Scatter(y=np.array(removed)*N, mode='lines+markers', name='removed', marker=dict(size=3), 
                          line=dict(color='rgba(31,119,180,0.5)', width=1))) # muted_blue

figC.add_trace(go.Scatter(y=infectrem, mode='lines+markers', name='cases', 
                          line=dict(color='rgba(44,160,44,0.7)', width=2))) # cooked_asparagus_green

figC.add_trace(go.Scatter(y=casesgrowth, mode='lines+markers', name='cases growth', 
                          fill='tozeroy', fillcolor='rgba(148,103,189,0.3)', 
                          line=dict(color='rgba(148,103,189,0.7)', width=2))) # muted_purple

figC.update_layout(autosize=False, width=900, height=600)
figC.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=5))
figC.update_yaxes(title_text='people')
figC.update_xaxes(title_text='days', range=[25, 85])
figC.show()

Growth rate of confirmed cases maximum (purple) corresponds to the inflection point of cases sigmoid (green).